# Lesson 3: Agentic Search

In [1]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [2]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?",
                       include_answer=True)

# print the answer
result["answer"]


'The new Blackwell GPU features fifth-generation NVLink, FP4 precision, and a massive computational power for AI. It includes the Grace-Blackwell superchip and supports up to 576 GPUs. Blackwell Ultra offers premium AI services.'

## Regular search

In [3]:
# choose location (try to change to your own city!)

city = "San Francisco"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

> Note: search was modified to return expected results in the event of an exception. High volumes of student traffic sometimes cause rate limit exceptions.

In [4]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results  


for i in search(query):
    print(i)

C:\Users\SPYROSDP\AppData\Local\Temp\ipykernel_34464\2852339428.py:6: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddg = DDGS()


https://zhidao.baidu.com/question/635787571195459284.html
https://zhidao.baidu.com/question/339397003791592205.html
https://zhidao.baidu.com/question/434865041797516004.html
https://zhidao.baidu.com/question/1443692925088265899.html
https://zhidao.baidu.com/question/186995866995574324.html
https://zhidao.baidu.com/question/1058113297817713219.html


In [5]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


> Note: This produces a long output, you may want to right click and clear the cell output after you look at it briefly to avoid scrolling past it.

In [6]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

Website: https://weather.com/weather/today/l/San+Francisco+CA?canonicalCityId=d060bcef6a904af38313393bd51e4c3c


<body><div class="appWrapper DaybreakLargeScreen LargeScreen lightTheme twcTheme DaybreakLargeScreen--appWrapper--ZkDop" id="appWrapper"><div class="region-sidebarNavigation regionSidebarNavigation"><div class="removeIfEmpty" id="WxuSidebarNavigation-sidebarNavigation-sidebar-nav-injection"><div aria-hidden="false" class="SidebarNavigation--sidebarWrapper--v+g78"><nav aria-label="Site Navigation Links" class="SidebarNavigation--sidebar--A5yHr" data-testid="sidebar-navigation"><div class="SidebarNavigation--sidebarHeader--bMAxo"><button class="SidebarNavigation--overlayControlButton--cSFvn openSidebarOverlayButton" tabindex="0" type="button"><span class="Icon--iconWrapper--vSeDL"><svg class="Icon--icon--ySD-o" fill="none" height="24" name="hamburger" viewbox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg">
<title>Hamburger</title>
<path d="M20 16C20.5523 16 21 16.44

In [7]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://weather.com/weather/today/l/San+Francisco+CA?canonicalCityId=d060bcef6a904af38313393bd51e4c3c


Recents Weather Forecasts Radar & Maps News & Media Products & Account Lifestyle Specialty Forecasts San Francisco, California High Wind Warning Weather Today in San Francisco, California 6:47 am 5:00 pm Radar Rain Rain possible after 5 PM. Hourly Weather - San Francisco, California Now Cloudy 2 pm Cloudy 3 pm Cloudy 4 pm Cloudy Don't Miss Travel 10 Day Weather - San Francisco, California Today Day Mostly cloudy skies. Slight chance of a shower late. High 67F. Winds S at 10 to 15 mph. Night Cloudy skies with periods of rain late. Low 62F. Winds SSE at 15 to 25 mph. Chance of rain 100%. Rainfall around a half an inch. Thu 13 Day Occasional rain tapering to a few showers late. High 64F. Winds SSW at 15 to 25 mph. Chance of rain 90%. Rainfall around a half an inch. Higher wind gusts possible. Night Partly to mostly cloudy. Low 54F. Winds S at 5 to 10 mph. Fri 14 Day A few passi

## Agentic Search

In [8]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1762978113, 'localtime': '2025-11-12 12:08'}, 'current': {'last_updated_epoch': 1762977600, 'last_updated': '2025-11-12 12:00', 'temp_c': 17.8, 'temp_f': 64.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 11.6, 'wind_kph': 18.7, 'wind_degree': 185, 'wind_dir': 'S', 'pressure_mb': 1016.0, 'pressure_in': 30.01, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 73, 'cloud': 100, 'feelslike_c': 17.8, 'feelslike_f': 64.0, 'windchill_c': 17.1, 'windchill_f': 62.9, 'heatindex_c': 17.2, 'heatindex_f': 62.9, 'dewpoint_c': 13.1, 'dewpoint_f': 55.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 2.4, 'gust_mph': 16.4, 'gust_kph': 26.4}}


In [9]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "San Francisco",
        "region": "California",
        "country": "United States of America",
        "lat": 37.775,
        "lon": -122.4183,
        "tz_id": "America/Los_Angeles",
        "localtime_epoch": 1762978113,
        "localtime": "2025-11-12 12:08"
    },
    "current": {
        "last_updated_epoch": 1762977600,
        "last_updated": "2025-11-12 12:00",
        "temp_c": 17.8,
        "temp_f": 64.0,
        "is_day": 1,
        "condition": {
            "text": "Overcast",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/122.png",
            "code": 1009
        },
        "wind_mph": 11.6,
        "wind_kph": 18.7,
        "wind_degree": 185,
        "wind_dir": "S",
        "pressure_mb": 1016.0,
        "pressure_in": 30.01,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 73,
        "cloud": 100,
        "feelslike_c": 17.8,
        "feelslike_f": 64.0,
        "windchill_c": 17.1,
        "

<img src="./google_sample.png" width="800" height="600">